In [111]:
# Librerías

import numpy as np

import math

from sklearn.metrics import accuracy_score

# Defino la función sigmoide

def sigmoide(x):
    return 1 / (1 + math.exp(-x))

# Programo una función que devuelva h1, h2 , .... , hn dado un x^(i), los pesos y bias 

def Capa_Oculta(x,coeficientes,cant_neuronas):
    
    
    # El orden estricto de los coeficientes es w_1_01 (bias), w_1_11, ... , w_1_n1 (todos los pesos de la primera neurona con 
    
    # el bias primero), w_1_02, w_1_12, ..... , w_1_n2 , ..... , w_1_ncant_neuronas . 
    
    cant_variables = x.shape[0]
    
    outputs = []
    
    for i in range(cant_neuronas):
        
        z = 0
        
        for j in range(cant_variables + 1):
            
            if j == 0 :
                
                z = z + coeficientes[i][j]
                
            else:
                
                z = z + coeficientes[i][j]*x[j-1]
                
        outputs.append(sigmoide(z))
    
    
    # Devuelvo h1, h2, .... , hn
    
    return outputs

# Programo otra función que a partir de h1 , h2, ...., hn  pesos y bias devuelva o:

def Capa_Output(outputs,coeficientes):
    
    
    # El orden estricto de los coeficientes es w_2_0 (bias), w_2_1, ... , w_2_cant_neuronas (todos los pesos con 
    
    # el bias primero).
    
    # Calculo el output
    
    z = 0
    
    for i in range(len(coeficientes)):
        
        if i == 0 :
            
            z = z + coeficientes[i]
            
        else:
            
            z = z + coeficientes[i]*outputs[i-1]
            
            
            
    
            
            
    output_final = sigmoide(z)
    
    # Devuelvo el output
    
    return output_final
    


In [114]:
# Programo propiamente la red neuronal


class Red_Neuronal_Basica():
    
    ## Asigno valores random  a cada peso y bias
    
    def __init__(self,cant_neuronas,cant_inputs):
        
        
        # Guardo las variables de la clase
        
        self.cant_neuronas = cant_neuronas
        
        self.cant_inputs = cant_inputs
        
        self.coeficientes = []
        
        
        # Inicializo los bias y pesos
        
        # Coeficientes_Capa_Oculta 
        
        
        
        for j in range(cant_neuronas):
            
            Aux = []
        
            for i in range(self.cant_inputs+1):
                
                A = np.random.normal(0,1)
            
                Aux.append(A)
            
            self.coeficientes.append(Aux)
            
        # Coeficientes_Capa_Output 
        
        Aux = []
        
        for i in range(self.cant_neuronas+1):
            
            A = np.random.normal(0,1)
            
            Aux.append(A)
            
        self.coeficientes.append(Aux)
        
        
    
        
    # Buscamos algunos buenos parámetros
        
    def train(self, x, y, alpha = 0.5,treshold = 0.00005, Reg = 0):
        
        # Consideramos el error cuadrático medio como función de costo:
        
        m , n = x.shape
        
        
        
        # Iniciamos el loop
        
        while True:
            
            diff = []
            
            # w_2_0
            
            coef_viejo = self.coeficientes[-1][0]
            
            # Inicializo la variable que me ayuda a sumar
            
            L = 0
            
            # Para cada dato
            
            for i in range(m) :
                
                H = Capa_Oculta(x[i],self.coeficientes[0:-1],self.cant_neuronas)
                
                o = Capa_Output(H,self.coeficientes[-1])
                
                L = L + (o - y[i])*o*(1-o)
                
                
            # Obtengo la dirección de máximo crecimiento
            
        
                
            L = (L * 2) / m + 2*Reg*self.coeficientes[-1][0]
            
            # Actualizo w_2_0
            
            self.coeficientes[-1][0] = self.coeficientes[-1][0] - alpha * L
            
            
            diff.append((self.coeficientes[-1][0] - coef_viejo)**2)
            
            
            # w_2_iésimos
            
            for j in range(self.cant_neuronas):
                
                coef_viejo = self.coeficientes[-1][j+1]
                
                # Inicializo la variable que me ayuda a sumar
            
                L = 0

                # Para cada dato

                for i in range(m) :

                    H = Capa_Oculta(x[i],self.coeficientes[0:-1],self.cant_neuronas)

                    o = Capa_Output(H,self.coeficientes[-1])

                    L = L + (o - y[i])*o*(1-o)*H[j]

                # Obtengo la dirección de máximo crecimiento

                L = (L * 2) / m + 2*Reg*self.coeficientes[-1][j+1]

                # Actualizo w_2_0

                self.coeficientes[-1][j+1] = self.coeficientes[-1][j+1] - alpha * L


                diff.append((self.coeficientes[-1][j+1] - coef_viejo)**2)
                
            # Bias neuronas de la capa oculta
                
            for j in range(self.cant_neuronas):
                    
                coef_viejo = self.coeficientes[j][0]
                
                # Inicializo la variable que me ayuda a sumar

                L = 0

                # Para cada dato

                for i in range(m) :

                    H = Capa_Oculta(x[i],self.coeficientes[0:-1],self.cant_neuronas)

                    o = Capa_Output(H,self.coeficientes[-1])

                    L = L + (o - y[i])*o*(1-o)*self.coeficientes[-1][j]*H[j]*(1-H[j])

                # Obtengo la dirección de máximo crecimiento

                L = (L * 2) / m + 2*Reg*self.coeficientes[j][0]

                # Actualizo w_2_0

                self.coeficientes[j][0] = self.coeficientes[j][0] - alpha * L


                diff.append((self.coeficientes[j][0] - coef_viejo)**2)
                    
                    
            # Pesos de la capa oculta:
                
            for k in range(self.cant_inputs):
                
                for j in range(self.cant_neuronas):

                    coef_viejo = self.coeficientes[j][k+1]

                    # Inicializo la variable que me ayuda a sumar

                    L = 0

                    # Para cada dato

                    for i in range(m) :

                        H = Capa_Oculta(x[i],self.coeficientes[0:-1],self.cant_neuronas)

                        o = Capa_Output(H,self.coeficientes[-1])

                        L = L + (o - y[i])*o*(1-o)*self.coeficientes[-1][j]*H[j]*(1-H[j])*x[i][k]

                    # Obtengo la dirección de máximo crecimiento

                    L = (L * 2) / m + 2*Reg*self.coeficientes[j][k+1]

                    # Actualizo w_2_0

                    self.coeficientes[j][k+1] = self.coeficientes[j][k+1] - alpha * L


                    diff.append((self.coeficientes[j][k+1] - coef_viejo)**2)
                

            
            dif_total = 0
            
            for i in diff:
                
                dif_total = dif_total + i
            
            if math.sqrt(dif_total) <= treshold:
                
                break
                
            else:
                
                pass
                
    # Hago un método que prediga
                
    def pred(self,x,treshold_1 = 0.5):
        
        m, n = x.shape
        
        Pred= []
        
        for i in range(m):
            
            H = Capa_Oculta(x[i],self.coeficientes[0:-1],self.cant_neuronas)
                
            o = Capa_Output(H,self.coeficientes[-1])
            
            
            
            if o >= treshold_1:
                
                Pred.append(1)
                
            else:
                
                Pred.append(0)
                
        return Pred